## Linear Regression with Gradient Descent

**Gradient of Cost Function:**

OLS:
$ \nabla_\theta J(\theta) = \frac{2}{m} X^\top (X\theta - y) $

Lasso Regression:
$ \nabla_\theta J(\theta) = \frac{2}{m} X^\top (X\theta - y) \;+\; \lambda \, \text{sign}(\theta^*) $

Ridge Regression:
$ \nabla_\theta J(\theta) = \frac{2}{m} X^\top (X\theta - y) \;+\; 2\lambda \theta^* $

ElasticNet:
$
\nabla_\theta J(\theta) = \frac{2}{m} X^\top (X\theta - y) 
\;+\; \lambda \alpha \, \text{sign}(\theta^*) 
\;+\; 2\lambda (1-\alpha) \theta^*
$

In [1]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error

In [2]:
class LinearRegressionGD:
    def __init__(
        self,
        lr=0.01,
        n_iters=1000,
        batch_size=None,
        regularization=False,
        regularization_method=None,
        lambda_=1.0,
        alpha=0.5
    ):
        """
        lr: learning rate
        n_iters: number of epochs
        batch_size:
            - None  -> Batch GD (full dataset)
            - 1     -> Stochastic GD
            - k     -> Mini-batch GD of size k
        regularization: regularization technique
        regularization_method:
            - L1: Lasso
            - L2: Ridge
            - Elastic: ElasticNet
        lambda_: regularization strength
        alpha: ElasticNet mixing parameter
        """
        self.lr = lr
        self.n_iters = n_iters
        self.batch_size = batch_size
        self.theta = None
        self.regularization = regularization
        self.regularization_method = regularization_method
        self.lambda_ = lambda_
        self.alpha = alpha

    def fit(self, X, y):
        ones = np.ones((X.shape[0], 1))
        X_b = np.hstack([ones, X])

        n_samples, n_features = X_b.shape
        self.theta = np.zeros((n_features, 1))

        batch_size = self.batch_size or n_samples

        for epoch in range(self.n_iters):
            idx = np.random.permutation(n_samples)
            X_b_shuffled = X_b[idx]
            y_shuffled = y[idx]

            for start in range(0, n_samples, batch_size):
                end = start + batch_size
                X_batch = X_b_shuffled[start:end]
                y_batch = y_shuffled[start:end]

                y_pred = X_batch @ self.theta

                grad = (2 / X_batch.shape[0]) * (X_batch.T @ (y_pred - y_batch))

                if self.regularization:
                    theta_reg = self.theta.copy()
                    theta_reg[0] = 0

                    if self.regularization_method == 'L1':
                        grad += self.lambda_ * np.sign(theta_reg)
                    elif self.regularization_method == 'L2':
                        grad += self.lambda_ * 2 * theta_reg
                    elif self.regularization_method == 'Elastic':
                        grad += (self.lambda_ * self.alpha * np.sign(theta_reg) + self.lambda_ * (1 - self.alpha) * 2 * theta_reg)

                self.theta -= self.lr * grad

            if epoch % 100 == 0:
                y_full_pred = X_b @ self.theta
                mse = np.mean((y_full_pred - y) ** 2)
                print(f"Epoch {epoch}/{self.n_iters} — Train MSE: {mse:.4f}")

    def predict(self, X):
        ones = np.ones((X.shape[0], 1))
        X_b = np.hstack([ones, X])
        return X_b @ self.theta

In [3]:
housing = fetch_california_housing()
X, y = housing.data, housing.target

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [5]:
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [6]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
print("Training shape:", X_train.shape, y_train.shape)
print("Test shape:", X_test.shape, y_test.shape)

Training shape: (16512, 8) (16512, 1)
Test shape: (4128, 8) (4128, 1)


In [8]:
model = LinearRegressionGD()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = np.mean((y_pred - y_test) ** 2)
print("Test MSE:", mse)

Epoch 0/1000 — Train MSE: 5.4312
Epoch 100/1000 — Train MSE: 0.7066
Epoch 200/1000 — Train MSE: 0.5950
Epoch 300/1000 — Train MSE: 0.5731
Epoch 400/1000 — Train MSE: 0.5583
Epoch 500/1000 — Train MSE: 0.5476
Epoch 600/1000 — Train MSE: 0.5398
Epoch 700/1000 — Train MSE: 0.5341
Epoch 800/1000 — Train MSE: 0.5299
Epoch 900/1000 — Train MSE: 0.5268
Test MSE: 0.5545795065308549


In [9]:
model = LinearRegressionGD(regularization=True, regularization_method='L1')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = np.mean((y_pred - y_test) ** 2)
print("Test MSE:", mse)

Epoch 0/1000 — Train MSE: 5.4312
Epoch 100/1000 — Train MSE: 1.0587
Epoch 200/1000 — Train MSE: 0.9522
Epoch 300/1000 — Train MSE: 0.9454
Epoch 400/1000 — Train MSE: 0.9476
Epoch 500/1000 — Train MSE: 0.9504
Epoch 600/1000 — Train MSE: 0.9476
Epoch 700/1000 — Train MSE: 0.9491
Epoch 800/1000 — Train MSE: 0.9508
Epoch 900/1000 — Train MSE: 0.9465
Test MSE: 0.9369758678060054


In [10]:
model = LinearRegressionGD(regularization=True, regularization_method='L2')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = np.mean((y_pred - y_test) ** 2)
print("Test MSE:", mse)

Epoch 0/1000 — Train MSE: 5.4312
Epoch 100/1000 — Train MSE: 0.8865
Epoch 200/1000 — Train MSE: 0.8081
Epoch 300/1000 — Train MSE: 0.8065
Epoch 400/1000 — Train MSE: 0.8065
Epoch 500/1000 — Train MSE: 0.8065
Epoch 600/1000 — Train MSE: 0.8065
Epoch 700/1000 — Train MSE: 0.8065
Epoch 800/1000 — Train MSE: 0.8065
Epoch 900/1000 — Train MSE: 0.8065
Test MSE: 0.8021077157095555


In [11]:
model = LinearRegressionGD(regularization=True, regularization_method='Elastic')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = np.mean((y_pred - y_test) ** 2)
print("Test MSE:", mse)

Epoch 0/1000 — Train MSE: 5.4312
Epoch 100/1000 — Train MSE: 0.9714
Epoch 200/1000 — Train MSE: 0.8890
Epoch 300/1000 — Train MSE: 0.8840
Epoch 400/1000 — Train MSE: 0.8860
Epoch 500/1000 — Train MSE: 0.8835
Epoch 600/1000 — Train MSE: 0.8847
Epoch 700/1000 — Train MSE: 0.8855
Epoch 800/1000 — Train MSE: 0.8869
Epoch 900/1000 — Train MSE: 0.8839
Test MSE: 0.8805722162049375


In [12]:
models = {
    "SGDRegressor": SGDRegressor(loss='squared_error', random_state=42),
    "Ridge": Ridge(solver='saga'),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet()
}

In [13]:
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"{name:10s} Test MSE: {mse:.4f}")

SGDRegressor Test MSE: 0.5506
Ridge      Test MSE: 0.5559
Lasso      Test MSE: 1.3107
ElasticNet Test MSE: 1.0442


/Users/kashyappatel/anaconda3/envs/ds605/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Linear Regression with Normal Equation

### OLS (Ordinary Least Squares):

$
J(\theta) \;=\; \frac{1}{\text{m}}\,\|y - X\theta\|_2^2 \;=\; \frac{1}{\text{m}}(y - X\theta)^\top (y - X\theta)
$

$
J(\theta)=\frac{1}{\text{m}}\big(y^\top y - 2 y^\top X\theta + \theta^\top X^\top X \theta\big)
$

* Gradient of $y^\top y$ wrt $\theta$ is 0.
* Gradient of $2 y^\top X\theta$ wrt $\theta$ is $2 X^\top y$.
    * $\nabla_\theta (A^\top \theta) = A$
* Gradient of $\theta^\top X^\top X \theta$ wrt $\theta$ is $2 X^\top X \theta$.
    * $\nabla_\theta (\theta^\top A \theta) = (A + A^\top)\theta$
    * if $A$ is symmetric ($X^\top X$ is symmetric) then $\nabla_\theta (\theta^\top A \theta) = 2 A \theta$

$
\nabla_\theta J(\theta) = \frac{2}{\text{m}}X^\top\big(X \theta - y \big)
$

$
-X^\top y + X^\top X \theta = 0 \quad\Longrightarrow\quad X^\top X \theta = X^\top y
$

$
\theta_{\text{OLS}} = (X^\top X)^{-1} X^\top y\
$

If $X^\top X$ is singular or nearly singular, replace inverse with the Moore–Penrose pseudoinverse:

$
\theta_{\text{OLS}} = X^\dagger y$, where $X^\dagger = (X^\top X)^\dagger X^\top
$

---

### Ridge Regression:

$
J(\theta) \;=\; \frac{1}{\text{m}}\|y - X\theta\|_2^2 \;+\; \lambda\,\|\theta\|_2^2
\;=\; \frac{1}{\text{m}}(y - X\theta)^\top (y - X\theta) + \lambda\theta^\top\theta
$

$
\nabla_\theta J(\theta) = \frac{2}{\text{m}}X^\top\big(X \theta - y \big) + 2 \lambda \theta
$

$
\frac{2}{\text{m}}X^\top\big(X \theta - y \big) + 2 \lambda \theta = 0 \quad\Longrightarrow\quad (\frac{1}{\text{m}}X^\top X + \lambda I)\theta = \frac{1}{\text{m}}X^\top y
$

$
I' = \begin{bmatrix} 0 & 0 \\ 0 & I_{d-1} \end{bmatrix},\quad
\theta = (X^\top X + \text{m}\lambda I')^{-1} X^\top y
$

In [14]:
class LinearRegressionNE:
    def __init__(self, regularization=None, lambda_=1.0):
        """
        regularization: 
            - None  -> OLS
            - 'L2'  -> Ridge
        lambda_: regularization strength
        """
        self.regularization = regularization
        self.lambda_ = lambda_
        self.theta = None

    def fit(self, X, y):
        ones = np.ones((X.shape[0], 1))
        X_b = np.hstack([ones, X])

        n_features = X_b.shape[1]
        m = X_b.shape[0]

        if self.regularization is None:
            self.theta = np.linalg.pinv(X_b.T @ X_b) @ (X_b.T @ y)

        elif self.regularization == 'L2':
            I = np.eye(n_features)
            I[0, 0] = 0
            self.theta = np.linalg.pinv(X_b.T @ X_b + m * self.lambda_ * I) @ (X_b.T @ y)

        else:
            raise ValueError("Only 'None' (OLS) and 'L2' (Ridge) are supported.")

    def predict(self, X):
        ones = np.ones((X.shape[0], 1))
        X_b = np.hstack([ones, X])
        return X_b @ self.theta

In [15]:
model = LinearRegressionNE()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = np.mean((y_pred - y_test) ** 2)
print("Test MSE:", mse)

Test MSE: 0.5558915986952413


In [16]:
model = LinearRegressionNE(regularization='L2')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = np.mean((y_pred - y_test) ** 2)
print("Test MSE:", mse)

Test MSE: 0.8021077157578121


In [17]:
models = {
    "OLS": LinearRegression(),
    "Ridge": Ridge(solver='cholesky')
}

In [18]:
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"{name:10s} Test MSE: {mse:.4f}")

OLS        Test MSE: 0.5559
Ridge      Test MSE: 0.5559
